<a href="https://colab.research.google.com/github/MohamedElgabryy/Spark/blob/main/Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq

!wget -q https://downloads.apache.org/spark/spark-3.2.4/spark-3.2.4-bin-hadoop2.7.tgz

!tar xf spark-3.2.4-bin-hadoop2.7.tgz

!mv spark-3.2.4-bin-hadoop2.7 /usr/local/spark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/usr/local/spark"

os.environ["PATH"] += ":/usr/local/spark/bin"

!pip install pyspark==3.2.4

!pip install findspark

from pyspark import SparkConf, SparkContext
import time

# Stop the existing SparkContext
SparkContext.getOrCreate().stop()

# Create a new SparkContext
conf = SparkConf().setAppName("MyApp")
sc = SparkContext(conf=conf)

data=sc.textFile('/content/drive/MyDrive/pagecounts-20160101-000000_parsed.out')

mv: cannot move 'spark-3.2.4-bin-hadoop2.7' to '/usr/local/spark/spark-3.2.4-bin-hadoop2.7': Directory not empty
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Define initial values for min, max, and total page size
min_page_size = float('inf')
max_page_size = 0
total_page_size = 0
count = 0

# Start the timer
start_time = time.time()

# Iterate over each line and update the min, max, and total page size
for line in data.toLocalIterator():
    fields = line.split(" ")
    if len(fields) == 4:
        page_size = int(fields[3])
        min_page_size = min(min_page_size, page_size)
        max_page_size = max(max_page_size, page_size)
        total_page_size += page_size
        count += 1

# Compute the average page size
avg_page_size = total_page_size / count
# End the timer
end_time = time.time()
# Calculate the execution time
execution_time = end_time - start_time

# Print the results
print("Min Page Size: {}".format(min_page_size))
print("Max Page Size: {}".format(max_page_size))
print("Average Page Size: {}".format(avg_page_size))
print("-------------------------------------------")
print("Execution Time: {} seconds".format(execution_time))

# Write the results to the output file
output_file_path = "page_size_stats.txt"

with open(output_file_path, 'w') as f:
    f.write("Min Page Size: {}\n".format(min_page_size))
    f.write("Max Page Size: {}\n".format(max_page_size))
    f.write("Average Page Size: {}\n".format(avg_page_size))
    f.write("-------------------------------------------\n")
    f.write("Execution Time: {} seconds\n".format(execution_time))

Min Page Size: 0
Max Page Size: 141180155987
Average Page Size: 132239.56957446598
Execution Time: 36.60955333709717 seconds


In [ ]:
# Define initial values
total_page_title = 0
non_english = 0
all_titles = 0

# Start the timer
start_time = time.time()

# Iterate over each line and update the total_page_title and non_english counts
for line in data.toLocalIterator():
    all_titles += 1
    fields = line.split(" ")
    page_title = fields[1]
    project_code = fields[0]

    if page_title.lower().startswith("the_") or page_title.lower() == "the":
        total_page_title += 1
        if project_code != "en":
            non_english += 1

# Print the results
print("Total Page Titles starting with 'The':", total_page_title)
print("Non-English Page Titles starting with 'The':", non_english)
print("Total Titles Processed:", all_titles)
print("-------------------------------------------")
print("Elapsed Time:", time.time() - start_time, "seconds")

# Write the results to the output file
output_file_path = "page_title_stats.txt"

with open(output_file_path, 'w') as f:
    f.write("Total Page Titles starting with 'The': {}\n".format(total_page_title))
    f.write("Non-English Page Titles starting with 'The': {}\n".format(non_english))
    f.write("Total Titles Processed: {}\n".format(all_titles))
    f.write("-------------------------------------------\n")
    f.write("Elapsed Time: {} seconds\n".format(time.time() - start_time))

Total Page Titles starting with 'The': 42018
Non-English Page Titles starting with 'The': 9262
Total Titles Processed: 3324129
-------------------------------------------
Elapsed Time: 39.44256258010864 seconds


In [ ]:
import re

# Initialize a set to store unique terms
unique_terms = set()

start_time = time.time()
# Iterate over each line in the dataset
for line in data.toLocalIterator():
    fields = line.split(" ")
    page_title = fields[1]

    # Normalize the page title
    normalized_title = page_title.lower().replace("_", " ")
    normalized_title = re.sub(r"[^a-zA-Z0-9\s]", "", normalized_title)

    # Split the normalized title into terms
    terms = normalized_title.split(" ")

    # Add each term to the set of unique terms
    for term in terms:
        if term:
            unique_terms.add(term)

# Get the number of unique terms
num_unique_terms = len(unique_terms)

# Print the results
print("Number of unique terms:", num_unique_terms)
print("Elapsed Time:", time.time() - start_time, "seconds")

# Write the results to the output file
output_file_path = "unique_terms_stats.txt"

with open(output_file_path, 'w') as f:
    f.write("Number of unique terms: {}\n".format(num_unique_terms))
    f.write("Elapsed Time: {} seconds\n".format(time.time() - start_time))


Number of unique terms: 1688531
Elapsed Time: 39.16520380973816 seconds


In [ ]:
total_the_titles = 0
titles = {}

start_time = time.time()

for line in data.toLocalIterator():
    line_fields = line.split(" ")
    title = line_fields[1].lower()

    if title in titles:
        titles[title] += 1
    else:
        titles[title] = 1

end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Print the elapsed time
print("Time taken for the execution:")
print(elapsed_time, "seconds")

# Print the number of tuples
print("The number of tuples are:", len(titles))

# Print each title and its count
for key, value in titles.items():
    print(key, value)

# Write the results to the output file
output_file_path = "title_counts_stats.txt"

with open(output_file_path, 'w') as f:
    f.write("Time taken for the execution: {} seconds\n".format(elapsed_time))
    f.write("The number of tuples are: {}\n".format(len(titles)))
    for key, value in titles.items():
        f.write("{}: {}\n".format(key, value))


Streaming output truncated to the last 5000 lines.
file:portrait_of_augustus,_27-20_bc,_from_the_excavation_sites_on_the_via_del_mare,_centrale_montemartini,_rome_(21926928069).jpg 1
file:portrait_of_bernard_lens_iii_by_p.p.lens.jpg 1
file:portrait_of_bernhard_siegfried_albinus;_1823_wellcome_l0004877.jpg 1
file:portrait_of_david_alfonso.jpg 1
file:portrait_of_dr._gachet.jpg 1
file:portrait_of_dr._harvey_w._wiley.jpg 1
file:portrait_of_euphimia_nosova_by_konstantin_somov.jpg 1
file:portrait_of_francesco_andreini_in_the_costume_of_capitano_spavento_-_ferrone_2014_fig4.jpg 1
file:portrait_of_fredrika_bremer_by_johan_gustaf_sandberg_1843.jpeg 1
file:portrait_of_friedrich_nietzsche.jpg 1
file:portrait_of_g._wither_after_j._payne,_1669_wellcome_l0008186.jpg 1
file:portrait_of_henri_matisse_1933_may_20.jpg 1
file:portrait_of_isaac_abrahamsz._massa_by_frans_hals.png 1
file:portrait_of_ishwari_sen_of_mandi.jpg 1
file:portrait_of_j.g.children._wellcome_l0013626.jpg 1
file:portrait_of_jacob_van_

KeyboardInterrupt: ignored

In [ ]:
combined_pages_dict = {}

start_time = time.time()

for line in data.toLocalIterator():
    fields = line.split(" ")
    project_code = fields[0]
    page_title = fields[1]
    page_hits = int(fields[2])
    page_size = int(fields[3])

    # Create a tuple for page data
    page_data = (project_code, page_hits, page_size)

    # Append the page data to the list of the corresponding title in the dictionary
    if page_title in combined_pages_dict:
        combined_pages_dict[page_title].append(page_data)
    else:
        combined_pages_dict[page_title] = [page_data]

combined_pages_dict = {title: page_list for title, page_list in combined_pages_dict.items() if len(page_list) > 1}

output_file_path = "file.txt"
with open(output_file_path, "w") as f:
    for title, page_list in combined_pages_dict.items():
        f.write("\n***************************************\n\n")
        f.write("Page Title: {}\n\n".format(title))
        print("\n***************************************\n")
        print(f"Page Title: {title}\n")
        for page_data in page_list:
            f.write("Project Code: {}\n".format(page_data[0]))
            f.write("Page Hits: {}\n".format(page_data[1]))
            f.write("Page Size: {}\n".format(page_data[2]))
            f.write("----------------\n")
            print("Project Code:", page_data[0])
            print("Page Hits:", page_data[1])
            print("Page Size:", page_data[2])
            print("----------------")

end_time = time.time()

elapsed_time = end_time - start_time

print("Elapsed Time:", elapsed_time, "seconds")

sc.stop()

Streaming output truncated to the last 5000 lines.
----------------
Project Code: ht
Page Hits: 1
Page Size: 12823
----------------
Project Code: hu
Page Hits: 4
Page Size: 411240
----------------
Project Code: ia
Page Hits: 1
Page Size: 14520
----------------
Project Code: id
Page Hits: 2
Page Size: 83523
----------------
Project Code: io
Page Hits: 1
Page Size: 11918
----------------
Project Code: is
Page Hits: 2
Page Size: 37254
----------------
Project Code: it
Page Hits: 3
Page Size: 140580
----------------
Project Code: ja
Page Hits: 10
Page Size: 977918
----------------
Project Code: jv
Page Hits: 1
Page Size: 14379
----------------
Project Code: ka
Page Hits: 1
Page Size: 20161
----------------
Project Code: kaa
Page Hits: 1
Page Size: 13638
----------------
Project Code: kk
Page Hits: 1
Page Size: 20907
----------------
Project Code: ku
Page Hits: 1
Page Size: 14656
----------------
Project Code: ky
Page Hits: 1
Page Size: 13607
----------------
Project Code: la
Page Hits: 1
P

KeyboardInterrupt: ignored